<a href="https://colab.research.google.com/github/Zhi-704/ERL/blob/master/Genetic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install jumanji
import jumanji
import jax.numpy as jnp
import jax
import random
import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
from time import sleep

In [ ]:
# Useful Code
'''
# Instantiate tetris environment using registry

env = jumanji.make('Tetris-v0', num_rows = 5, time_limit = 1000)

# Reset your (jit-able) environment
key = jax.random.PRNGKey(1)
state, timestep = jax.jit(env.reset)(key)

def convert_state(self, state):
  '''
  # Convert state into observation variable that can be passed into neural network
  # PARAM -
  # state: current game state
  # RETURNS - observation variable
  '''

  grid = state.grid_padded.flatten().tolist()
  tetromino = state.tetromino_index.flatten().tolist()
  # print(grid)
  # print(tetromino)
  obs_variable = np.asarray(grid+tetromino)
  # print("Current input dimensions:")
  # print(np.shape(obs_variable))
  # print(obs_variable)

  return obs_variable

def policy(self,state):
  '''
  # Takes state from environment and returns an action that has the highest q value using epsilon_greedy
  # PARAM -
  # state: current game state
  # RETURNS - action
  '''
  # Matches state into 105, array
  state_array = self.convert_state(state)
  # Convert to into tensorflow tensor
  state_input = tf.convert_to_tensor(state_array[None, :], dtype=tf.float32)
  # Grabs Q values for all possible actions in current state
  action_q = self.q_net(state_input)

  action_mask = np.array(state.action_mask).flatten()

  valid_q_values = action_q * action_mask
  # Set elements that are zero to -999
  valid_q_values = np.where(valid_q_values == 0, -999, valid_q_values)

  print("ACTION MASK")
  print(action_mask)
  print("CORRESPONDING Q VALUES")
  print(valid_q_values)
  # print(np.shape(valid_q_values))
  true_indices = self.get_valid_actions(state)
  print(true_indices)
  if len(true_indices) == 0:
    return False, False
  else:
    action_index = np.argmax(valid_q_values, axis=1)[0]
    # Convert action into acceptable type
    array = np.array(state.action_mask)
    rotation, col_index = np.unravel_index(action_index, array.shape)
    action = [rotation, col_index]
    return action, action_index

def get_valid_actions(self,state):
  array = np.array(state.action_mask)
  # Flatten the array
  flattened_array = array.flatten()
  # Get the indices of true values, which are the number of valid actions in the current state
  true_indices = np.where(flattened_array)[0]

  return true_indices

def random_action(self, state):
  array = np.array(state.action_mask)
  true_indices = self.get_valid_actions(state)
  if len(true_indices) == 0:
    return False
  random_index = np.random.choice(true_indices)
  # Convert the random index to two-dimensional coordinates
  rotation, col_index = np.unravel_index(random_index, array.shape)
  return [rotation,col_index]
  '''

In [ ]:
'''
GA()
   initialize population
   find fitness of population

   while (termination criteria is reached) do
      parent selection
      crossover with probability pc
      mutation with probability pm
      decode and fitness calculation
      survivor selection
      find best
   return best
'''

In [ ]:
# https://github.com/robertjankowski/ga-openai-gym/blob/master/scripts/ga/individual.py

class Population:
  def __init__(self, individual, pop_size, max_generation, p_mutation, p_crossover, p_inversion):
    self.pop_size = pop_size
    self.max_generation = max_generation
    self.p_mutation = p_mutation
    self.p_crossover = p_crossover
    self.p_inversion = p_inversion
    self.old_population = [copy.copy(individual) for _ in range(pop_size)]
    self.new_population = []

In [ ]:
class Population:

  def __init__(self, pop_size, p_mutation, p_crossover, p_selection):
    self.pop_size = pop_size
    self.p_mutation = p_mutation
    self.p_crossover = p_crossover
    self.p_selection = p_selection

class Individual:
  def __init__(self, input_size, hidden_size, output_size):
    self.nn = self.build_model(input_size, hidden_size, output_size)
    self.fitness = 0.0
    self.weights_biases: np.array = None

  def build_model(self, input_size, hidden_size, output_size):
    # 128, 64, 40 input, hidden,output

    model = Sequential()
    # Adds fully connected layer with 128 units and uses rectified linear unit activation function. he_uniform initliazes weight of layer
    model.add(Dense(128, input_dim = input_size, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(hidden_size, activation='relu', kernel_initializer='he_uniform'))
    # 40 actions so 40 different outputs
    model.add(Dense(output_size, activation='linear', kernel_initializer='he_uniform'))

    # opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer = 'adam')
    return model

In [ ]:
# https://www.youtube.com/watch?app=desktop&v=nhT56blfRpE&t=85s
from random import choices, randint, randrange, random
from typing import List, Optional, Callable, Tuple

Genome = List[int]

# Generate single genome for one solution
def generate_genome(length: int) -> Genome:
  return choices([0,1], k = length)

# Generate how every many genomes in a population
def generate_population(size: int, genome_length:int) -> Population:
  return [generate_genome(genome_length) for _ in range(size)]

def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
  if len(a) != len(b):
      raise ValueError("Genomes a and b must be of same length")

  length = len(a)
  if length < 2:
      return a, b

  p = randint(1, length - 1)
  return a[0:p] + b[p:], b[0:p] + a[p:]


def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
  for _ in range(num):
    index = randrange(len(genome))
    genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
  return genome


def population_fitness(population: Population, fitness_func: FitnessFunc) -> int:
  return sum([fitness_func(genome) for genome in population])


def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
  return choices(
    population=population,
    weights=[fitness_func(gene) for gene in population],
    k=2
  )


def sort_population(population: Population, fitness_func: FitnessFunc) -> Population:
  return sorted(population, key=fitness_func, reverse=True)


def genome_to_string(genome: Genome) -> str:
  return "".join(map(str, genome))


def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc):
  print("GENERATION %02d" % generation_id)
  print("=============")
  print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
  print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
  sorted_population = sort_population(population, fitness_func)
  print(
      "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
  print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                            fitness_func(sorted_population[-1])))
  print("")

  return sorted_population[0]


def run_evolution(
      populate_func: PopulateFunc,
      fitness_func: FitnessFunc,
      fitness_limit: int,
      selection_func: SelectionFunc = selection_pair,
      crossover_func: CrossoverFunc = single_point_crossover,
      mutation_func: MutationFunc = mutation,
      generation_limit: int = 100,
      printer: Optional[PrinterFunc] = None) \
      -> Tuple[Population, int]:
  population = populate_func()

  for i in range(generation_limit):
    population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

    if printer is not None:
      printer(population, i, fitness_func)

    if fitness_func(population[0]) >= fitness_limit:
      break

    next_generation = population[0:2]

    for j in range(int(len(population) / 2) - 1):
      parents = selection_func(population, fitness_func)
      offspring_a, offspring_b = crossover_func(parents[0], parents[1])
      offspring_a = mutation_func(offspring_a)
      offspring_b = mutation_func(offspring_b)
      next_generation += [offspring_a, offspring_b]

    population = next_generation

  return population, i